In [ ]:
import torch
import torchvision
import videoseal 
from IPython.display import HTML, display
from base64 import b64encode
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import logging
logging.getLogger("matplotlib.image").setLevel(logging.ERROR)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fps = 24 


# Load the VideoSeal Model
The videoseal library provides pretrained models for embedding and extracting watermarks. 

In [ ]:
# Load the VideoSeal model
model = videoseal.load("videoseal")

# Set the model to evaluation mode and move it to the selected device
model.eval()
model.to(device)

# Load and Preprocess the Video
We will process a single video. For demonstration, the video is trimmed to the first 3 seconds.

In [ ]:
# Path to the input video
video_path = "./assets/videos/2.mp4"

# Read the video and convert to tensor format
video, _, _ = torchvision.io.read_video(video_path, output_format="TCHW")

# Normalize the video frames to the range [0, 1] and trim to 1 second
video = video[:fps * 5].float() / 255.0


# Videoseal Embedder: Embed the Watermark
The model embeds a watermark into the video frames. This step generates the watermarked frames (imgs_w) and returns the random watermark message (msgs) so you know which message was embedded. 

In [ ]:
# Perform watermark embedding
outputs = model.embed(video, is_video=True)

# Extract the results
video_w = outputs["imgs_w"]  # Watermarked video frames
msgs = outputs["msgs"]      # Watermark messages


In [ ]:
# Compute the difference between watermarked and original frames
diff = video - video_w

# Normalize the difference for visualization
min_vals = diff.view(diff.shape[0], diff.shape[1], -1).min(dim=2, keepdim=True)[0].view(diff.shape[0], diff.shape[1], 1, 1)
max_vals = diff.view(diff.shape[0], diff.shape[1], -1).max(dim=2, keepdim=True)[0].view(diff.shape[0], diff.shape[1], 1, 1)
normalized_images = (diff - min_vals) / (max_vals - min_vals)

# Display The Watermark

In [ ]:
def display_videos_side_by_side(videos, titles):
    """
    Display multiple videos side by side in a table format in a Jupyter Notebook.

    Args:
    - videos: List of video tensors (torch.Tensor).
    - titles: List of column titles corresponding to the videos.
    """
    # Prepare video HTML strings
    video_htmls = []
    for video in videos:

        video = video.permute(0, 2, 3, 1).cpu().numpy()  # Convert to numpy format
                
        # resize and display 2 secs only 
        video = video[:fps*2]

        _, C, H, W = video.shape
        new_H, new_W = 512, int((512 / H) * W)
        new_H, new_W = (new_H // 2) * 2, (new_W // 2) * 2  # Ensure even dimensions
        # Resize the video
        video = F.resize(video, (new_H, new_W))


        
        
        fig, ax = plt.subplots()
        img = ax.imshow(video_np[0])  # Display the first frame

        # Remove axis ticks and labels
        ax.axis('off')

        def update(frame):
            img.set_data(video_np[frame])
            return img,

        ani = FuncAnimation(fig, update, frames=len(video_np), interval=1000 // fps)
        plt.close(fig)
        video_htmls.append(ani.to_jshtml())
    
    # Create table with titles as headers and videos as content
    table_html = '<table style="width:100%; text-align:center; border-collapse: collapse;">'
    table_html += '<tr>'  # Row for headers
    for title in titles:
        table_html += f'<th style="border: 1px solid black; padding: 5px;">{title}</th>'
    table_html += '</tr>'
    table_html += '<tr>'  # Row for videos
    for video_html in video_htmls:
        table_html += f'<td style="border: 1px solid black; padding: 5px;">{video_html}</td>'
    table_html += '</tr>'
    table_html += '</table>'
    
    return HTML(table_html)

# Display the videos side by side
videos = [video, video_w, normalized_images]
titles = ["Original Video", "Watermarked Video", "Watermark"]
display(display_videos_side_by_side(videos, titles))


# Videoseal Extractor: Extract the Watermark 

In [ ]:
from videoseal.evals.metrics import bit_accuracy

msg_extracted = model.extract_message(video_w)

bit_accuracy_ = bit_accuracy(msg_extracted, msgs).nanmean().item()
print(f"Bit Accuracy: {bit_accuracy_:.3f}")



## Test VideoSeal Robustness to H264 Codec

In [ ]:
import pandas as pd 
from videoseal.augmentation import H264

def compress_and_eval(video_w, msgs, compression_levels):
    """
    Test the watermark resistance to H.264 compression.
    
    Args:
    - video_w (torch.Tensor): Watermarked video frames.
    - msgs (torch.Tensor): Original watermark messages.
    - compression_levels (list): List of CRF values for compression.
    
    Returns:
    - results (list): List of dictionaries containing CRF, Bit Accuracy, and Detection Time.
    """
    results = []
    h264 = H264()  # H.264 compression augmentation

    for crf in compression_levels:
        # Apply H.264 compression
        video_compressed, _ = h264(video_w, crf=crf)

        # Detect watermarks in the compressed video        
        outputs = model.detect(video_compressed, is_video=True)

        # Compute bit accuracy
        preds = outputs["preds"]
        bit_preds = preds[:, 1:]  # Extract watermark bits
        bit_acc = bit_accuracy(bit_preds, msgs).nanmean().item()

        # Store results
        results.append({
            "CRF": crf,
            "Bit Accuracy": round(bit_acc, 3),
        })

    return results

# Test the watermark resistance at different CRF levels
compression_levels = [15, 23, 30, 32, 35, 38, 40, 60]  # CRF values for H.264 compression
results = compress_and_eval(video_w, msgs, compression_levels)

# Display results in a table
df_results = pd.DataFrame(results)
display(HTML("<h3>Watermark Resistance to Compression</h3>"))
display(df_results)